In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from math import ceil
import torch
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.append('../training_utils/')
from data_utils import get_folders, get_class_weights

# utils specific to quantization
from utils import train, quantize, initial_scales
torch.cuda.is_available()

In [ ]:
torch.backends.cudnn.benchmark = True

# Create data iterators

In [ ]:
batch_size = 32

In [ ]:
train_folder, val_folder = get_folders()

train_iterator = DataLoader(
    train_folder, batch_size=batch_size, num_workers=4,
    shuffle=True, pin_memory=True
)

val_iterator = DataLoader(
    val_folder, batch_size=64, num_workers=4,
    shuffle=False, pin_memory=True
)

# number of training samples
train_size = len(train_folder.imgs)
train_size

# Model

In [ ]:
from get_densenet121 import get_model

In [ ]:
# w[j]: 1/number_of_samples_in_class_j
# decode: folder name to class name (in human readable format)
w, decode = get_class_weights(val_folder.class_to_idx)

In [ ]:
model, criterion, optimizer = get_model(class_weights=torch.FloatTensor(w/w.sum()))

# load pretrained model, accuracy ~85%
model.load_state_dict(torch.load('../densenet/model121.pytorch_state'))

In [ ]:
# number of params in the model
count_params(model)

#### Keep copy of full precision kernels

In [ ]:
# copy all full precision kernels of the model
all_fp_kernels = [
    Variable(kernel.data.clone(), requires_grad=True) 
    for kernel in optimizer.param_groups[2]['params']
]
# all_fp_kernels - kernel tensors of all convolutional layers 
# (with the exception of the first conv layer)

# optimizer for updating only all_fp_kernels
optimizer_fp = optim.SGD(all_fp_kernels, lr=1e-3)

#### initial quantization 

In [ ]:
# scaling factors for each quantized layer
scaling_factors = []

In [ ]:
# these kernels will be quantized
all_kernels = [kernel for kernel in optimizer.param_groups[2]['params']]

In [ ]:
 for k, k_fp in zip(all_kernels, all_fp_kernels):
    
    # choose initial scaling factors 
    w_p_initial, w_n_initial = initial_scales(k_fp.data)
    scaling_factors += [(w_p_initial, w_n_initial)]
    
    # do quantization, this uses hyperparameter t defined in utils.py file
    k.data = quantize(k_fp.data, w_p_initial, w_n_initial)

# Train

In [ ]:
n_epochs = 10
n_batches = ceil(train_size/batch_size)
n_batches

In [ ]:
%%time
all_losses, _ = train(
    model, scaling_factors, criterion, 
    optimizer, optimizer_fp,
    train_iterator, n_epochs, n_batches,
    val_iterator, validation_step=531, n_validation_batches=80
)
# epoch logloss    accuracy     top5_accuracy time  (first value: train, second value: val)

# Loss/epoch plots

In [ ]:
epochs = [x[0] for x in all_losses]
plt.plot(epochs, [x[1] for x in all_losses], label='train');
plt.plot(epochs, [x[2] for x in all_losses], label='val');
plt.legend();
plt.xlabel('epoch');
plt.ylabel('loss');

In [ ]:
plt.plot(epochs, [x[3] for x in all_losses], label='train');
plt.plot(epochs, [x[4] for x in all_losses], label='val');
plt.legend();
plt.xlabel('epoch');
plt.ylabel('accuracy');

In [ ]:
plt.plot(epochs, [x[5] for x in all_losses], label='train');
plt.plot(epochs, [x[6] for x in all_losses], label='val');
plt.legend();
plt.xlabel('epoch');
plt.ylabel('top5_accuracy');

# Save

In [ ]:
model.cpu();
torch.save(model.state_dict(), 'model121_ternary_quantization.pytorch_state')